# Personal Information
Name: Jasper Braakman

StudentID: 12418781

Email: [jasper.braakman@student.uva.nl](youremail@student.uva.nl)

Submitted on: 22-03-2024

GitHub: 

In [82]:
# Imports
import geopandas as gpd
import pandas as pd
import numpy as np

# Raster libraries
import rasterio
from rasterio.features import rasterize

# Random
import random

### General Description of "Zaanstad boorpunten data":
- Name: Zaanstad boorpunten data
- Type: Categorical variable

### Categories:
- Below background value (<=AW)
- Above background value (>AW)
- Above intervention value (>I)
- Above in between value (>T)

### Transformations:
1. Remove rows with NaN values from within date or soil test.
2. Filtering by Measurement Year: Remove all rows that are measured before 2008. This is done because the methodology used to sample and evaluate soil measurements was only regulated in the Netherlands after a law was passed in 2008.
3. Include only points that have 10 or more neighboring points within a radius of 25 meters. This is done because later on we will use these 10 neighbors as an additional variable.

### Conclusion:
After transformations we see that we are left with about 1/6th of the data that is almost evenly split. When training the model still a decision will have to be taken if the data has to be further trimmed to allow equally sized categories.

In [5]:
# Load the specific layer from the geopackage
gdf = gpd.read_file("../Data/Bodemloket_20231009_Antea_Group.gpkg", layer="TBL_WFS_GEO_BORING_ONDERZOEK")

In [6]:
# Describtion of the raw data
print("General Description of Zaanstad boorpunten data:")
print("Number of observations:", len(gdf))
print("Categories and their counts:")
print(gdf['TOETS_WBB'].value_counts())

General Description of Zaanstad boorpunten data:
Number of observations: 119958
Categories and their counts:
>AW     31502
>I      23138
<=AW    13006
>T       8886
Name: TOETS_WBB, dtype: int64


In [7]:
# Transformation 1: Removing NULL values from DATUM_RAP and TOETS_WBB columns
gdf = gdf.dropna(subset=['DATUM_RAP'])
gdf = gdf.dropna(subset=['TOETS_WBB'])

In [8]:
# Transformation 2: Filtering by Measurement Year
# Convert DATUM_RAP to datetime format
gdf['DATUM_RAP'] = pd.to_datetime(gdf['DATUM_RAP'], format='%d-%m-%Y')

# Remove all measurements before 2008
gdf = gdf[gdf['DATUM_RAP'].dt.year >= 2008]

In [9]:
# Transformation 3: Changing categories of TOETS_WBB
gdf.loc[gdf['TOETS_WBB'] != '>I', 'TOETS_WBB'] = '<I'

In [10]:
# Transformation 4: Spatial Filtering
# Calculate number of neighboring points within a radius of 25 meters
gdf['neighbors'] = gdf.geometry.apply(lambda x: len(gdf[gdf.geometry.within(x.buffer(25))]) - 1)

In [11]:
# Filter points with 10 or more neighbors
zaanstad_data_filtered = gdf[gdf['neighbors'] >= 10]

In [12]:
# Describtion of the raw data
print("General Description of Zaanstad boorpunten data:")
print("Number of observations:", len(zaanstad_data_filtered))
print("Categories and their counts:")
print(zaanstad_data_filtered['TOETS_WBB'].value_counts())

General Description of Zaanstad boorpunten data:
Number of observations: 18525
Categories and their counts:
<I    11370
>I     7155
Name: TOETS_WBB, dtype: int64


In [13]:
# Define the file path for the new geopackage
output_gpkg = "../Data/cleaned_data_zaanstad.gpkg"

# Save the filtered GeoDataFrame to the new geopackage
zaanstad_data_filtered.to_file(output_gpkg, driver="GPKG")

print("Filtered data saved to:", output_gpkg)

Filtered data saved to: cleaned_data_zaanstad.gpkg


### General Description of "Bodem Kwaliteits Kaart data":
- Name: BKK
- Type: Categorical variable

### Categories:
- AW-2000 (low risk zone)
- Wonen (medium risk zone)
- Industrie (high risk zone)

### Transformations:
For each point within the soil sample dataset we check in which risk zone this soil sample is.

### Conclusions
As shown in the print statement the likelyhood of finding a polluted point within a risk zone increases with the higher level of risk associated with the risk zone. This might indicate that the expert knowledge imbedded in the model through this variable might contribute to predicting polution.

In [49]:
# Load the points dataset containing "bodem kwaliteits kaart" and "TOETS_WBB" variables
points_gdf = gpd.read_file("../Data/cleaned_data_zaanstad.gpkg", layer="cleaned_data_zaanstad")

# Load the raster dataset representing "bodem kwaliteits kaart"
raster_path = "../Data/bbk_Zaanstad.tif"  # Change to the actual file path

# Open the raster dataset
with rasterio.open(raster_path) as src:
    # Read the raster data
    raster_data = src.read(1)

    # Get the affine transformation parameters
    transform = src.transform

# Categories
categories = {
    'AW_2000': 138,
    'Wonen': 252,
    'Industrie': 254,
    'Onbekend': 142
}

# Define the conditions for TOETS_WBB
toets_wbb_conditions = {
    'Greater than I': '>I',
    'Less than I': '<I',
}

# Initialize counts for each category and condition
category_counts = {category: {condition: 0 for condition in toets_wbb_conditions} for category in categories}

# Iterate over each point in the GeoDataFrame
for index, point in points_gdf.iterrows():
    # Extract the x, y coordinates of the point
    x = point.geometry.x
    y = point.geometry.y
    
    try:
        # Convert the x, y coordinates to row, column indices in the raster
        row, col = src.index(x, y)
        
        # Get the pixel value at the corresponding row, column indices
        pixel_value = raster_data[row, col]
        
        # Access the value in the TOETS_WBB column for the current point
        toets_wbb_value = point['TOETS_WBB']
        
        # Check if the pixel value matches any category
        for category, category_value in categories.items():
            if pixel_value == category_value:
                # Check the conditions on TOETS_WBB
                for condition_name, condition in toets_wbb_conditions.items():
                    if condition == toets_wbb_value:
                        # Increment count for the current category and condition
                        category_counts[category][condition_name] += 1
    
    except IndexError:
        # Skip this point and continue with the next
        continue

# Print the counts for each category and condition
for category, condition_counts in category_counts.items():
    print(f"Category: {category}")
    for condition, count in condition_counts.items():
        print(f" - {condition}: {count}")

Category: AW_2000
 - Greater than I: 25
 - Less than I: 111
Category: Wonen
 - Greater than I: 862
 - Less than I: 2164
Category: Industrie
 - Greater than I: 6217
 - Less than I: 9065
Category: Onbekend
 - Greater than I: 0
 - Less than I: 0


### General Description of "Basisregistratie Gewaspercelen":
- Name: Basisregistratie Gewaspercelen
- Type: Categorical variable

### Categories:
- Agricultural fields
- No aggricultural fields

### Transformations:
For each point within the soil sample dataset we check wether it is taken inside an agricultural field or not.

### Conclusion
From the analysis we see that most points lie outside of the aggricultural fields, indicating that the variable will most likely not contribute much to the model. However, later in the reseach we can adjust this variable by drawing buffers around the aggricultural fields, this could be done since it can logically be assumed that the influence of the fields is larger than the ground themself (take for example pestisides or fertalizer that contaminates the ground water). With this adjusted variable we could reevaluate the importance of this variable.

In [81]:
# Load the polygon shapefile
polygons = gpd.read_file("../Data/Zaanstad_Clipped_Databronnen.gpkg", layer="BRP_Gewaspercelen")

# Load the point shapefile
points = gpd.read_file("../Data/cleaned_data_zaanstad.gpkg", layer="cleaned_data_zaanstad")

# Perform spatial join
points_in_polygons = gpd.sjoin(points, polygons, predicate='within')

# Count points inside and outside polygons
inside_count = len(points_in_polygons)
outside_count = len(points) - inside_count

# Count points belonging to each category
category_counts = points_in_polygons['TOETS_WBB'].value_counts()

print("Points outside polygons:", outside_count)
print("Points inside polygons:", inside_count)
print("Points belonging to each category:")
print(category_counts)

Points outside polygons: 18497
Points inside polygons: 28
Points belonging to each category:
<I    26
>I     2
Name: TOETS_WBB, dtype: int64


### General Description of "Rivieren":
- Name: Rivers
- Type: Categorical variable

### Categories:
- Within 10m buffer of a river
- Outside 10m buffer of a river

### Transformations:
For each point within the soil sample dataset we check wether it is within the a buffer of 10 meters around water bodies.

### Conclusion:
From the print statements we can see that the data indicates that around water bodies we can expect on avarage a more clean soil.

In [60]:
# Read the shapefiles into GeoDataFrames
points_gdf = gpd.read_file("../Data/cleaned_data_zaanstad.gpkg", layer="cleaned_data_zaanstad")

# Water files
lines_gdf = gpd.read_file("../Data/Zaanstad_Clipped_Databronnen.gpkg", layer="oppervlaktewaterlichaam_zaanstad")

# Create 25m buffers around lines and polygons
lines_buffer = lines_gdf.copy()  # Copy the GeoDataFrame
lines_buffer['geometry'] = lines_buffer.geometry.buffer(10)  # Apply buffer operation

# Spatial join between points and lines
points_within_lines = gpd.sjoin(points_gdf, lines_buffer, how='inner', predicate='within')

# Count points within the buffer by category
points_within_buffer_count = points_within_lines.groupby('TOETS_WBB').size()

# Filter out points within the buffer
points_outside_buffer = points_gdf[~points_gdf.index.isin(points_within_buffer.index)]

# Count points outside the buffer by category
points_outside_buffer_count = points_outside_buffer.groupby('TOETS_WBB').size()

# Perform spatial join between points and buffer to find points outside the buffer
points_outside_buffer = gpd.sjoin(points_gdf, lines_buffer, how='left', predicate='within')
points_outside_buffer = points_outside_buffer[points_outside_buffer.index_right.isnull()]

# Count points outside the buffer by category
points_outside_buffer_count = points_outside_buffer.groupby('TOETS_WBB').size()

print("Points outside the 25m buffer:")
print(points_outside_buffer_count)

print("Points within 25m buffer:")
print(points_within_buffer_count)

Points outside the 25m buffer:
TOETS_WBB
<I    9775
>I    5772
dtype: int64
Points within 25m buffer:
TOETS_WBB
<I    2087
>I    1819
dtype: int64


### General Description of "Wegen":
- Name: Roads
- Type: Categorical variable

### Categories:
- Within 25m of a road
- Not within 25m of a road

### Transformations:
For each point within the soil sample dataset we check wether it is within the a buffer of 10 meters around roads.

### Conclusion:
From the print statements we can see that the data indicates that around roads we can expect on avarage a poluted soil.

In [64]:
# Read the shapefiles into GeoDataFrames
points_gdf = gpd.read_file("../Data/cleaned_data_zaanstad.gpkg", layer="cleaned_data_zaanstad")

# Water files
lines_gdf = gpd.read_file("../Data/Zaanstad_Clipped_Databronnen.gpkg", layer="wegvakken")

# Create 25m buffers around lines and polygons
lines_buffer = lines_gdf.copy()  # Copy the GeoDataFrame
lines_buffer['geometry'] = lines_buffer.geometry.buffer(10)  # Apply buffer operation

# Spatial join between points and lines
points_within_lines = gpd.sjoin(points_gdf, lines_buffer, how='inner', predicate='within')

# Count points within the buffer by category
points_within_buffer_count = points_within_lines.groupby('TOETS_WBB').size()

# Filter out points within the buffer
points_outside_buffer = points_gdf[~points_gdf.index.isin(points_within_buffer.index)]

# Count points outside the buffer by category
points_outside_buffer_count = points_outside_buffer.groupby('TOETS_WBB').size()

# Perform spatial join between points and buffer to find points outside the buffer
points_outside_buffer = gpd.sjoin(points_gdf, lines_buffer, how='left', predicate='within')
points_outside_buffer = points_outside_buffer[points_outside_buffer.index_right.isnull()]

# Count points outside the buffer by category
points_outside_buffer_count = points_outside_buffer.groupby('TOETS_WBB').size()

print("Points outside the 10m buffer:")
print(points_outside_buffer_count)

print("Points within 10m buffer:")
print(points_within_buffer_count)

Points outside the 10m buffer:
TOETS_WBB
<I    7306
>I    4898
dtype: int64
Points within 10m buffer:
TOETS_WBB
<I    5416
>I    2812
dtype: int64


### General Description of "spoorwegen":
- Name: Railroad
- Type: Categorical variable

### Categories:
- Within 10m of a Railroad
- Not within 10m of a Railroad

### Transformations:
For each point within the soil sample dataset we check wether it is within the a buffer of 10 meters around railroads.

### Conclusion:
From the print statements we can see that the data indicates that around railroads we can expect on avarage a poluted soil.

In [66]:
# Read the shapefiles into GeoDataFrames
points_gdf = gpd.read_file("../Data/cleaned_data_zaanstad.gpkg", layer="cleaned_data_zaanstad")

# Water files
lines_gdf = gpd.read_file("../Data/Zaanstad_Clipped_Databronnen.gpkg", layer="spooras")

# Create 25m buffers around lines and polygons
lines_buffer = lines_gdf.copy()  # Copy the GeoDataFrame
lines_buffer['geometry'] = lines_buffer.geometry.buffer(10)  # Apply buffer operation

# Spatial join between points and lines
points_within_lines = gpd.sjoin(points_gdf, lines_buffer, how='inner', predicate='within')

# Count points within the buffer by category
points_within_buffer_count = points_within_lines.groupby('TOETS_WBB').size()

# Filter out points within the buffer
points_outside_buffer = points_gdf[~points_gdf.index.isin(points_within_buffer.index)]

# Count points outside the buffer by category
points_outside_buffer_count = points_outside_buffer.groupby('TOETS_WBB').size()

# Perform spatial join between points and buffer to find points outside the buffer
points_outside_buffer = gpd.sjoin(points_gdf, lines_buffer, how='left', predicate='within')
points_outside_buffer = points_outside_buffer[points_outside_buffer.index_right.isnull()]

# Count points outside the buffer by category
points_outside_buffer_count = points_outside_buffer.groupby('TOETS_WBB').size()

print("Points outside the 10m buffer:")
print(points_outside_buffer_count)

print("Points within 10m buffer:")
print(points_within_buffer_count)

Points outside the 10m buffer:
TOETS_WBB
<I    11254
>I     7103
dtype: int64
Points within 10m buffer:
TOETS_WBB
<I    528
>I    225
dtype: int64


### General Description of "ibis":
- Name: Industrial Areas
- Type: Categorical variable

### Categories:
- Within Industrial Areas
- Outside Industrial Areas

### Transformations:
For each point within the soil sample dataset we check wether it is taken inside an industrial area or not.

### Conclusion
From the data we can conclude that on avarage the soil is more clean in industrial areas than in non industrial areas. This might be counter intuative. However, this can be explained given the strick laws for industrial areas and the fact that we only use use recent soil samples (from 2008 onwards).

In [68]:
# Load the polygon shapefile
polygons = gpd.read_file("../Data/Zaanstad_Clipped_Databronnen.gpkg", layer="ibis_zaanstad")

# Load the point shapefile
points = gpd.read_file("../Data/cleaned_data_zaanstad.gpkg", layer="cleaned_data_zaanstad")

# Perform spatial join
points_in_polygons = gpd.sjoin(points, polygons, predicate='within')

# Count points inside and outside polygons
inside_count = len(points_in_polygons)
outside_count = len(points) - inside_count

# Count points belonging to each category
category_counts = points_in_polygons['TOETS_WBB'].value_counts()

print("Points outside polygons:", outside_count)
print("Points inside polygons:", inside_count)
print("Points belonging to each category:")
print(category_counts)

Points inside polygons: 2119
Points outside polygons: 16406
Points belonging to each category:
<I    1400
>I     719
Name: TOETS_WBB, dtype: int64


### General Description of "Surrounding Points variable":
- Name: Surrounding Points variable
- Type: Continues variable (fraction of poluted points to non-poluted points within its neighbourhood)

### Transformations:
For each point within the soil sample dataset we calculate the number of poluted points to non poluted points.

In [79]:
# Load the GeoPackage file
gdf = gpd.read_file("../Data/cleaned_data_zaanstad.gpkg", layer="cleaned_data_zaanstad")

# Function to select random points within each buffer
def select_random_points_within(points, n=10):
    selected_points = random.sample(list(points.iterrows()), min(n, len(points)))
    polluted_count = sum(1 for _, point in selected_points if point['TOETS_WBB'] == '>I')
    return polluted_count / n

# Select random points within each buffer and calculate fraction polluted
fraction_poluted_list = []
for index, row in gdf.iterrows():
    point_geometry = row.geometry
    buffer = point_geometry.buffer(25)
    points_within = gdf[gdf.geometry.within(buffer)]
    fraction_poluted = select_random_points_within(points_within_buffer)
    fraction_poluted_list.append(fraction_poluted)

# Add fraction_poluted to the GeoDataFrame
gdf['fraction_poluted'] = fraction_poluted_list

# Save the new dataset to a GeoPackage file
output_gpkg = "../Data/fraction_poluted.gpkg"
gdf.to_file(output_gpkg, driver="GPKG")

In [80]:
# Load the GeoPackage file with fraction_poluted
gdf = gpd.read_file("../Data/fraction_poluted.gpkg")

# Filter points with fraction_poluted above 0.5
high_fraction_poluted = gdf[gdf['fraction_poluted'] > 0.5]

# Calculate the fraction of points with label '>I' and '<I' among high fraction_poluted
total_points = len(high_fraction_poluted)
label_I_count = len(high_fraction_poluted[high_fraction_poluted['TOETS_WBB'] == '>I'])
label_not_I_count = len(high_fraction_poluted[high_fraction_poluted['TOETS_WBB'] == '<I'])

# Calculate the fractions
fraction_I = label_I_count / total_points
fraction_not_I = label_not_I_count / total_points

print("Fraction of points with label '>I' among high fraction_poluted:", fraction_I)
print("Fraction of points with label '<I' among high fraction_poluted:", fraction_not_I)

Fraction of points with label '>I' among high fraction_poluted: 0.38956078930617444
Fraction of points with label '<I' among high fraction_poluted: 0.6104392106938256
